## define functions

- contains requered number -> ensure only specific images are pull
- get image -> pull the require and compress image
- scrapp_image_description -> iterate for each of the products
- grucci -> iterate scrap_image

### scrapper

In [10]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO

from openpyxl import load_workbook
import re

import threading
import time

In [11]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf


In [12]:
class TimeoutException(Exception):
    pass

def create_soup(url, timeout=5):
    def fetch_soup():
        nonlocal soup, error
        try:
            ua = UserAgent()
            header = {'User-Agent': str(ua.chrome)}

            # Send an HTTP request to the URL
            response = requests.get(url, headers=header)
            soup = BeautifulSoup(response.content, 'html.parser')
        except Exception as e:
            error = e

    soup = None
    error = None

    thread = threading.Thread(target=fetch_soup)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        raise TimeoutException("Timeout while fetching the URL")
    if error:
        raise error

    return soup

In [13]:
def contains_required_number(line):
    
    keeping = ['001_100','004_100','005_100']
    # 001: Prod
    # 004: back
    # 005: 3/4 image
    return any(num in line for num in keeping)

In [14]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.find_all(attrs={"data-image-size": "small-retina"})

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        i+=1 
        if (i == len(item_list_1)) | (i == 1):
            continue
        elif not(contains_required_number(item['srcset'])) :
            continue
        else:
            images = images + [item['srcset']]
        
        i += 1

    return images

In [15]:
def get_description(prod_soup):
    
    description_list = prod_soup.find_all(attrs={"class": "product-detail"} )

    #Get description
    description = ''
    
    for a in description_list[0].ul.find_all('li'):
        description = description + ' ' + a.text
        

    return description_list[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description

    

In [16]:
def get_price(prod_soup):

    text = prod_soup.select('span.price')[1].text


    # Remove any non-numeric characters except for ',' and '.'
    cleaned_text = re.sub(r'[^\d,\.]', '', text)
    
    # Replace comma with a period if there's no period already (to handle decimal part)
    if ',' in cleaned_text and '.' not in cleaned_text:
        cleaned_text = cleaned_text.replace(',', '.')

    elif ',' not in cleaned_text and '.' in cleaned_text:
        cleaned_text = cleaned_text.replace('.', '')

    elif ',' in cleaned_text and '.' in cleaned_text:
        # If both ',' and '.' are present, keep only the period as the decimal separator
        cleaned_text = cleaned_text.replace('.', '')
        cleaned_text = cleaned_text.replace(',', '.')
    
    # Convert the string to a float
    number = float(cleaned_text)
    
    return number

In [17]:
def gucci(category_url,n_products, Testing):

    soup = create_soup(category_url)
    item_list = soup.select('article a')


    br = 0
    for item in item_list:
        if br == n_products:
            break

        retry_attempts = 3
        while retry_attempts > 0:
            try:
                # Scrape product details
                try:
                    prod_url = 'https://www.gucci.com' + item['href']
                    prod_soup = create_soup(prod_url)
                except TimeoutException:
                    print(f"Timeout fetching product {br}. Retrying in 5 seconds...")
                    time.sleep(5)
                    retry_attempts -= 1
                    continue
                except Exception as e:
                    print(f"Error fetching product {br}: {e}")
                    break


                prod_images_links = get_image_links(prod_soup=prod_soup)
                prod_description = get_description(prod_soup=prod_soup)
                prod_price = get_price(prod_soup=prod_soup)
                Clothing_type = soup.select('span.is-notlink')[0].text

                #prod_name = 'GUCCI_' + str(br)

                print(f'Starting product {br}')
                
                time.sleep(3)
                # Links to image, load to blob and return prod_images_names

                time.sleep(1)
                conn, cursor = SQLf.sql_db_functions.connect_sql()

                SQLf.sql_db_functions.insert_description_image_to_db(
                    conn=conn,
                    cursor=cursor,
                    brand='Gucci',
                    descript=prod_description,
                    price=prod_price,
                    prod_link = prod_url,
                    Clothing_type = Clothing_type,
                    images_links=prod_images_links,
                    Testing = Testing
                )

                conn.close()
                cursor.close()
                time.sleep(10)
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Error processing product {br}: {e}")
                #break  # Exit the retry loop if an exception occurs

        if retry_attempts == 0:
            print(f"Failed to process product {br} after 3 attempts. Exiting function.")
            print(f'failed link:')
            retry_attempts = 3


        br += 1



## Execute Scrapper

In [18]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women/tops-and-shirts-for-women-c-women-readytowear-tops-and-shirts'

gucci(url,35,False)

Timeout fetching product 0. Retrying in 5 seconds...
Starting product 0
inserting
Brand_Prod_id generated: 1
Data inserted successfully to DB
Starting product 1
inserting
Brand_Prod_id generated: 2
Data inserted successfully to DB
Starting product 2
inserting
Brand_Prod_id generated: 3
Data inserted successfully to DB
Timeout fetching product 3. Retrying in 5 seconds...
Starting product 3
inserting
Brand_Prod_id generated: 4
Data inserted successfully to DB
Starting product 4
inserting
Brand_Prod_id generated: 5
Data inserted successfully to DB
Starting product 5
inserting
Brand_Prod_id generated: 6
Data inserted successfully to DB
Starting product 6
inserting
Brand_Prod_id generated: 7
Data inserted successfully to DB
Starting product 7
inserting
Brand_Prod_id generated: 8
Data inserted successfully to DB
Starting product 8
inserting
Brand_Prod_id generated: 9
Data inserted successfully to DB
Starting product 9
inserting
Brand_Prod_id generated: 10
Data inserted successfully to DB
Sta